In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# creo una variabile che inizializzo con il sito su cui mi interessa fare scraping
w_url = 'https://it.wikipedia.org/wiki/Demografia_d%27Italia'

# creo una variabile e tramite il comando requests.get(nome_variabile_precedente).text lo trasformo in testo
wiki_page = requests.get(w_url).text

# creo un oggetto per il parsing della pagina html
soup = BeautifulSoup(wiki_page, 'html.parser')

# recupero le tabelle dalla pagina parsata passando come argomento il tag 'table'
tabelle = soup.find_all('table')

# creo un oggetto pandas
data_list = pd.read_html(str(tabelle[1]))
df = pd.DataFrame(data_list[0])
display(df)

Anno  Popolazione 1º gennaio          Natalità  \
                 Anno  Popolazione 1º gennaio              Nati   
0                1862              26.328.000           991.000   
1                1863              26.507.000         1.026.000   
2                1864              26.712.000         1.000.000   
3                1865              26.915.000         1.024.000   
4                1866              27.131.000         1.044.000   
..                ...                     ...               ...   
160          2022[24]              59.030.133           392.598   
161          2023[24]              58.850.717                 -   
162              Anno  Popolazione 1º gennaio          Natalità   
163              Anno  Popolazione 1º gennaio              Nati   
164  * = dati stimati        * = dati stimati  * = dati stimati   

                              Mortalità                      Saldo naturale  \
            Tasso (‰)             Morti         Tasso (‰)    Saldo naturale   
0                 375           815.000               309          +176.000   
1                 386           824.000               310          +202.000   
2                 373           802.000               299          +198.000   
3                 379           812.000               300          +212.000   
4                 383           798.000               293          +246.000   
..                ...               ...               ...               ...   
160                67           713.499               121          -320.901   
161                 -                 -                 -                 -   
162          Natalità         Mortalità         Mortalità    Saldo naturale   
163         Tasso (‰)             Morti         Tasso (‰)    Saldo naturale   
164  * = dati stimati  * = dati stimati  * = dati stimati  * = dati stimati   

    Tasso di fecondità totale[12][23]  
    Tasso di fecondità totale[12][23]  
0                                 493  
1                                 490  
2                                 490  
3                                 491  
4                                 491  
..                                ...  
160                               124  
161                               NaN  
162         Tasso di fecondità totale  
163         Tasso di fecondità totale  
164                  * = dati stimati  

[165 rows x 8 columns]

In [ ]:
anno_str        = df['Anno']['Anno'].to_numpy()[:-4]
popolazione_str = df["Popolazione 1º gennaio"]["Popolazione 1º gennaio"].to_numpy()[:-4]
nati_str        = df["Natalità"]["Nati"].to_numpy()[:-4]
morti_str       = df["Mortalità"]["Morti"].to_numpy()[:-4]
display(df)

Anno  Popolazione 1º gennaio          Natalità  \
                 Anno  Popolazione 1º gennaio              Nati   
0                1862              26.328.000           991.000   
1                1863              26.507.000         1.026.000   
2                1864              26.712.000         1.000.000   
3                1865              26.915.000         1.024.000   
4                1866              27.131.000         1.044.000   
..                ...                     ...               ...   
160          2022[24]              59.030.133           392.598   
161          2023[24]              58.850.717                 -   
162              Anno  Popolazione 1º gennaio          Natalità   
163              Anno  Popolazione 1º gennaio              Nati   
164  * = dati stimati        * = dati stimati  * = dati stimati   

                              Mortalità                      Saldo naturale  \
            Tasso (‰)             Morti         Tasso (‰)    Saldo naturale   
0                 375           815.000               309          +176.000   
1                 386           824.000               310          +202.000   
2                 373           802.000               299          +198.000   
3                 379           812.000               300          +212.000   
4                 383           798.000               293          +246.000   
..                ...               ...               ...               ...   
160                67           713.499               121          -320.901   
161                 -                 -                 -                 -   
162          Natalità         Mortalità         Mortalità    Saldo naturale   
163         Tasso (‰)             Morti         Tasso (‰)    Saldo naturale   
164  * = dati stimati  * = dati stimati  * = dati stimati  * = dati stimati   

    Tasso di fecondità totale[12][23]  
    Tasso di fecondità totale[12][23]  
0                                 493  
1                                 490  
2                                 490  
3                                 491  
4                                 491  
..                                ...  
160                               124  
161                               NaN  
162         Tasso di fecondità totale  
163         Tasso di fecondità totale  
164                  * = dati stimati  

[165 rows x 8 columns]

In [ ]:
# conversione stringhe - interi
# prende un array di stringhe e lo converte in un array di interi a 64 bit
# np.zeros_like crea un nuovo array delle medesime dimensioni dell'array data_str, ma con tutti gli elementi inizializzati a 0
def conversione_stringhe_interi(data_str: np.array(str)) -> np.array(np.int64):
  data = np.zeros_like(data_str)

# ciclo for per rimpiazzare i punti con una stringa vuota
  for i in range(data_str.size):
    data[i] = np.int64( data_str[i].replace('.', '') )
  return data

# conversione stringhe - anni (indice 0 = 1862, i++)
def conversione_anni(data_str: np.array(str)) -> np.array(np.int64):
  anni  = np.zeros_like(data_str)
  a0 = 1862
  for i in range(anni.size):
    anni[i] = a0
    a0 += 1
  return anni

# riconversione delle variabili
anni        = conversione_anni(anno_str)
popolazione = conversione_stringhe_interi(popolazione_str)
nati        = conversione_stringhe_interi(nati_str)
morti       = conversione_stringhe_interi(morti_str)

# costruzione del dataframe con le seguenti colonne
demo_df = pd.DataFrame(index = range(anni.size),  columns=['anno', 'nati', 'morti', "popolazione"]).fillna(0)
demo_df['anno'] = anni
demo_df['nati'] = nati
demo_df['morti'] = morti
demo_df['popolazione'] = popolazione

# creazione del file csv
demo_df.to_csv('demo_italia_1862_2021.csv')
display(demo_df)

anno     nati   morti popolazione
0    1862   991000  815000    26328000
1    1863  1026000  824000    26507000
2    1864  1000000  802000    26712000
3    1865  1024000  812000    26915000
4    1866  1044000  798000    27131000
..    ...      ...     ...         ...
156  2018   439747  633133    59937769
157  2019   420084  634417    59816673
158  2020   404892  740317    59641488
159  2021   400249  701346    59236213
160  2022   392598  713499    59030133

[161 rows x 4 columns]

In [ ]:
copia_df = demo_df.copy()
display(copia_df)
n = copia_df.isna()
display(n)

anno     nati   morti popolazione
0    1862   991000  815000    26328000
1    1863  1026000  824000    26507000
2    1864  1000000  802000    26712000
3    1865  1024000  812000    26915000
4    1866  1044000  798000    27131000
..    ...      ...     ...         ...
156  2018   439747  633133    59937769
157  2019   420084  634417    59816673
158  2020   404892  740317    59641488
159  2021   400249  701346    59236213
160  2022   392598  713499    59030133

[161 rows x 4 columns]

anno   nati  morti  popolazione
0    False  False  False        False
1    False  False  False        False
2    False  False  False        False
3    False  False  False        False
4    False  False  False        False
..     ...    ...    ...          ...
156  False  False  False        False
157  False  False  False        False
158  False  False  False        False
159  False  False  False        False
160  False  False  False        False

[161 rows x 4 columns]

No charts were generated by quickchart
